In [ ]:
# !pip install emoji

In [ ]:
import pandas as pd
a = pd.DataFrame([[1,10,3],[5,6,7]])
max(a.max())

In [ ]:
import csv
import emoji
from keras import backend as K
from keras import optimizers
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.engine import Layer
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Add, Average, Maximum, Concatenate
from keras.layers import Dropout, Input, TimeDistributed, PReLU
from keras.models import Sequential, Model, load_model, save_model, model_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.utils import to_categorical
from nltk.tokenize import TweetTokenizer
import numpy as np
import pandas as pd
import re
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import tensorflow as tf
import tensorflow_hub as hub
import time

from matplotlib import pyplot as plt
from IPython.display import clear_output

In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

#### Elmo layer

In [ ]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

#     def compute_mask(self, inputs, mask=None):
#         return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

#### Path vars & load

In [ ]:
path_prefix='../input/'

In [ ]:
TRAIN_FILE = path_prefix + 'train.txt'
DEV_FILE =  path_prefix + 'dev.txt'
TEST_FILE =  path_prefix + 'test.txt'

TURNS_NAMES = ["turn1", "turn2", "turn3"]
LABEL = ["label"]
CONCATENATED_TURNS = "turns"
def parse_file(file_path):
    output_dict = dict()
    with open(file_path, newline='\n', encoding='utf8') as csvfile:
        return pd.read_csv(csvfile, sep="\t")

In [ ]:
MODEL_CHECKPOINT = '3_elmo-turns'

In [ ]:

# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
train_data = parse_file(TRAIN_FILE)
dev_data = parse_file(DEV_FILE)
test_data = parse_file(TEST_FILE)

In [ ]:
print(np.shape(train_data))

In [ ]:
def concatenate_turns(df, delim="fullstop"):
    turns = [("%s %s %s %s %s" %
                 (row[TURNS_NAMES[0]], delim,
                  row[TURNS_NAMES[1]], delim,
                  row[TURNS_NAMES[2]])).lower()
                 for index, row in df.iterrows()]
    df[CONCATENATED_TURNS] = pd.Series(turns, index=df.index)
    return df

In [ ]:
def emoticons_replace(df):
    for index, row in df.iterrows():
        for turn in range(3):
          turns = emoji.demojize(row[TURNS_NAMES[turn]])
          # remove delimiters ":"  (:smiley: -> smiley)
          for emoj in re.findall(":\w*:", turns):
              turns  = turns.replace(emoj, emoj[1:-1]).replace("_", " ")
          df.at[index, TURNS_NAMES[turn]] = turns
    return df

In [ ]:
train = emoticons_replace(train_data)
dev = emoticons_replace(dev_data)
test = emoticons_replace(test_data)

In [ ]:
max_sentence = [163,82,189]
### angry: [1 0 0 0]
### happy: [0 1 0 0]
### others: [0 0 1 0]
### sad: [0 0 0 1]
labels = {0: 'angry',
          1: 'happy',
          2: 'others',
          3: 'sad'}

In [ ]:
### angry: [1 0 0 0]
### happy: [0 1 0 0]
### others: [0 0 1 0]
### sad: [0 0 0 1]
Y_train = pd.get_dummies(train[LABEL]).as_matrix()
# for i, t in enumerate(train[LABEL].iterrows()):
#     if t[1]['label']=='others':
#         print(Y_train[i])
#         break
Y_dev = pd.get_dummies(dev[LABEL]).as_matrix()
Y_test = pd.get_dummies(test[LABEL]).as_matrix()

#### Model Callback

In [ ]:
class Metrics(Callback):
    def __init__(self, test_X, test_Y, tolerance):
        self.test_X = test_X
        self.test_Y = test_Y
        self.max_f1 = 0
        self.f1_prev = 0
        self.tolerance = tolerance
        self.decreasing_times = 0
        
        
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
        self.i = 0
        self.x = []
        
        self.f1s_test = []
        self.f1s_val = []
        self.losses = []
        self.val_losses = []
        
        self.logs = []
        self.fig = plt.figure()
    
    def plot_losses(self, f1_val,f1_test, logs):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.f1s_test.append(f1_test)
        self.f1s_val.append(f1_val)
        self.i += 1
        
#         clear_output(wait=True)
        
        plt.subplot(2,1,1)
        plt.plot(self.x, self.losses, label="train_loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.subplot(2,1,2)
        plt.plot(self.x, self.f1s_val, label="f1_val " + '{:.2f}'.format(max(self.f1s_val)))
        plt.plot(self.x, self.f1s_test, label="f1_test " + '{:.2f}'.format(max(self.f1s_test)))
        
        plt.legend(loc=0)

        plt.show();

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict([self.validation_data[:3][i] for i in range(3)]))).round()
        val_targ = self.validation_data[3]

        _val_f1 = f1_score(val_targ, val_predict, average='micro')
        _val_recall = recall_score(val_targ, val_predict, average='micro')
        _val_precision = precision_score(val_targ, val_predict, average='micro')

        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print("— val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
        predicts = self.model.predict(self.test_X)
        test_predict = (np.asarray(predicts)).round()
        f1 = print_metrics_predicted(val_predict, val_targ, MODEL_CHECKPOINT+"-results")
        print("#############\nF1 test:\n#############")
        f_test = print_metrics_predicted(test_predict, self.test_Y, MODEL_CHECKPOINT+"-results")
        self.plot_losses(f1, f_test, logs)
        if f_test > self.max_f1:
            self.max_f1 = f_test
            self.model.save(MODEL_CHECKPOINT)
        if f_test < self.f1_prev:
            self.decreasing_times += 1
            if self.decreasing_times > self.tolerance:
                self.model.stop_training = True
        else:
            self.decreasing_times = 0
        self.f1_prev = f_test
        return
 
# metrics = Metrics()

def print_metrics_predicted(predicts,Y,filename):
    tp =[0,0,0,0]
    fp =[0,0,0,0]
    fn =[0,0,0,0]
    for i,pred in enumerate(predicts):
        p = np.argmax(pred)
        y = np.argmax(Y[i])
        if p == y:
            tp[p] += 1
        else:
            fp[p] +=1
            fn[y] +=1
    prec = sum(tp)/(sum(tp+fp)+np.finfo(float).eps)
    rec = sum(tp)/(sum(tp+fn)+np.finfo(float).eps)
    with open(filename, "w") as f:
      print("F1 all")
      f1_all = 2*prec*rec/(prec+rec+np.finfo(float).eps)
      print(f1_all) 
      f.write(str(f1_all))
      print("***")
      for i in range(4):
          print("F1 %s: " % labels[i])
          f.write("\n%s: " % labels[i])
          prec = tp[i]/(tp[i]+fp[i]+np.finfo(float).eps)
          rec = tp[i]/(tp[i]+fn[i]+np.finfo(float).eps)
          f1 = 2*prec*rec/(prec+rec+np.finfo(float).eps)
          print(f1)
          f.write(str(f1))
          print("****")
      tp.pop(2)
      fp.pop(2)
      fn.pop(2)
      print("F1 happy angry sad")
      f.write("\nF1 happy angry sad: ")
      prec = sum(tp)/(sum(tp+fp)+np.finfo(float).eps)
      rec = sum(tp)/(sum(tp+fn)+np.finfo(float).eps)
      f1= 2*prec*rec/(prec+rec+np.finfo(float).eps)
      f.write(str(f1))
      print(f1)
      return f1


##### Model params


In [ ]:
# vocabulary_size = len(tokenizer.word_counts) + 1

epochs = 10
embed_dim = 256
lstm_out = 128
batch_size = 128
drop_out = 0.3
loss_fct = 'binary_crossentropy'
activation_fct = 'softmax'
optimizer = "Adam-0.01"

parameters = """Epochs:%s\nEmbed_dim: %s\nLstm_out: %s\nBatch size: %s\nDrop_out: %s
Loss_fct: %s\nActivaion_fct: %s\nOptimizer: %s\n
""" %(str(epochs), str(embed_dim), str(lstm_out), str(batch_size), str(drop_out), loss_fct,
      activation_fct, optimizer)

###### Modele esuate

In [ ]:
# input_layer = Input(shape=(max_sentence,), dtype='int32')

# embedding_layer = Embedding(vocabulary_size,
#                             embed_dim,
#                             weights=[embedding_matrix],
#                             input_length=max_sentence,
#                             trainable=False)(input_layer)
# bi_lstm = Bidirectional(LSTM(lstm_out))(embedding_layer)
# dropout = Dropout(0.4)(bi_lstm)
# dense = Dense(128,activation=activation_fct)(dropout)
# dropout = Dropout(0.2)(dense)
# dense = Dense(4,activation=activation_fct)(dropout)
# adam = optimizers.Adam(lr=0.01)
# rmsprop = optimizers.RMSprop(lr=0.005)#, rho=0.9, epsilon=None, decay=0.0)
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# model = Model(inputs=[input_layer], outputs=dense)
# model.compile(loss = loss_fct, optimizer=adam, metrics = ['accuracy'])
# print(model.summary())

In [ ]:
# input_layer = Input(shape=(1,), dtype="string")
# embedding = ElmoEmbeddingLayer()(input_layer)
# # bilstm = Bidirectional(LSTM(lstm_out), input_shape=(1024,))(embedding)dr = Dropout(0.4)(embedding)
# dropout = Dropout(0.4)(embedding)
# dense = Dense(128,activation=activation_fct)(dropout)
# dropout = Dropout(0.2)(dense)
# dense = Dense(4,activation=activation_fct)(dropout)

# adam = optimizers.Adam(lr=0.01)
# rmsprop = optimizers.RMSprop(lr=0.005)#, rho=0.9, epsilon=None, decay=0.0)
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# model = Model(inputs=[input_layer], outputs=dense)
# model.compile(loss = 'binary_crossentropy', optimizer=adam, metrics = ['accuracy'])
# print(model.summary())

In [ ]:
# input_text = Input(shape=(1,), dtype="string")
# embedding = ElmoEmbeddingLayer()(input_text)
# # bilstm = Bidirectional(LSTM(lstm_out), input_shape=(1024,))(embedding)dr = Dropout(0.4)(embedding)
# dense = Dense(1024, activation='relu')(embedding)
# dense = Dense(512, activation='relu')(dense)
# dense = Dropout(0.15)(dense)
# dense = Dense(128, activation='relu')(dense)
# dense = Dropout(0.1)(dense)
# pred = Dense(4, activation='softmax')(dense)

# model = Model(inputs=[input_text], outputs=pred)
# adam = optimizers.Adam(lr=0.003)
# model.compile(loss='mean_squared_error', optimizer= adam, metrics=['accuracy'])
# model.summary()

##### Model

#### Model architecture


In [ ]:
input_layer1 = Input(shape=(1,), dtype="string")
embedding1 = ElmoEmbeddingLayer()(input_layer1)
input_layer2 = Input(shape=(1,), dtype="string")
embedding2 = ElmoEmbeddingLayer()(input_layer2)
input_layer3 = Input(shape=(1,), dtype="string")
embedding3 = ElmoEmbeddingLayer()(input_layer3)

merged = Average()([embedding1, embedding2, embedding3])

dropout = Dropout(0.4)(merged)
# dense = Dense(256,activation='relu',
#                 kernel_regularizer=regularizers.l2(0.01))(dropout)
# dropout = Dropout(0.3)(dense)
dense = Dense(128,activation='relu',
                kernel_regularizer=regularizers.l2(0.001))(dropout)
dropout = Dropout(0.2)(dense)
dense = Dense(64,activation='relu',
                kernel_regularizer=regularizers.l2(0.0001))(dropout)
dropout = Dropout(0.2)(dense)
dense = Dense(4,activation='softmax')(dropout)

adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)
sgd = optimizers.SGD(lr=1, decay=1e-6, momentum=0.9, nesterov=True)

model = Model(inputs=[input_layer1, input_layer2, input_layer3], outputs=dense)
model.compile(loss = 'categorical_crossentropy', optimizer=rmsprop, metrics = ['accuracy'])
print(model.summary())
# with open(MODEL_CHECKPOINT + ".json","w") as f:
#   f.write(model.to_json())

In [ ]:
# train_turns = train[TURNS_NAMES].T.values
# dev_turns = dev[TURNS_NAMES].T
# test_turns = test[TURNS_NAMES].T
# model.predict([test[TURNS_NAMES[i]] for i in range(3)])
# Y_test

In [ ]:
model.fit([train[TURNS_NAMES[i]] for i in range(3)], Y_train, epochs=100, verbose=1,
          batch_size=256,
          validation_data=([dev[TURNS_NAMES[i]] for i in range(3)], Y_dev),
          callbacks= [ Metrics([test[TURNS_NAMES[i]] for i in range(3)], Y_test, 3),
#                        ModelCheckpoint(MODEL_CHECKPOINT, monitor='val_loss', verbose=1, save_best_only=True, mode='max')
                    ])
# F1 test:
# #############
# F1 all
# 0.882555817752768
# ***
# F1 angry: 
# 0.49165402124430946
# ****
# F1 happy: 
# 0.6054054054054052
# ****
# F1 others: 
# 0.939195710455764
# ****
# F1 sad: 
# 0.638830897703549
# ****
# F1 happy angry sad
# 0.5705847607796809

In [ ]:
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
# predicts = (np.asarray(model.predict(test[TURNS_CONCAT][:3]))).round()

In [ ]:
# print_metrics_predicted(predicts,test[LABEL],"elmo1.txt")

In [ ]:
# with open(MODEL_CHECKPOINT + ".json", "r") as f:
#   model = model_from_json(f.read(), custom_objects={'ElmoEmbeddingLayer':ElmoEmbeddingLayer()})
  
# model.load_weights(MODEL_CHECKPOINT)

In [ ]:
# loaded_model = load_model(MODEL_CHECKPOINT, custom_objects={'ElmoEmbeddingLayer':ElmoEmbeddingLayer()})

In [ ]:
# predicts = (np.asarray(model.predict(test[CONCATENATED_TURNS][:3]))).round()

In [ ]:
# print(predicts[:10])

In [ ]:
# print_metrics_predicted(predicts, Y_test, filename=MODEL_CHECKPOINT+"-results")